In [ ]:
import os
import random
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.models import resnet18, ResNet18_Weights
from collections import defaultdict
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import requests
import re
import warnings
warnings.filterwarnings('ignore')

# ==================== ИНИЦИАЛИЗАЦИЯ ====================

# Очищаем память GPU
torch.cuda.empty_cache()

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ==================== КОНСТАНТЫ И ГИПЕРПАРАМЕТРЫ ====================

TRAIN_PQ = "/content/train_dataset.parquet"
TEST_PQ  = "/content/test_dataset.parquet"
TRAIN_IMG_DIR = "train_images"
TEST_IMG_DIR  = "test_images"

TRAIN_CSV = "train.csv"
TEST_CSV  = "test.csv"
CKPT_NAME = "improved_model"
CKPT_PATH = f"{CKPT_NAME}.pth"
USE_LOG_TARGET = True

# Оптимизированные гиперпараметры
IMG_SIZE = 224
N_IMAGES_PER_ITEM = 4
BATCH_SIZE = 32  # Увеличили батч
EPOCHS = 20      # Увеличили эпохи
LR = 1e-3        # Увеличили learning rate
WEIGHT_DECAY = 1e-6
VAL_SIZE = 0.15

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == "cuda":
    torch.cuda.manual_seed(SEED)

# ==================== ЗАГРУЗКА ДАННЫХ ====================

def get_direct_file_link(mailru_file_url: str) -> str:
    """Получение прямой ссылки для скачивания с Mail.ru"""
    resp = requests.get(mailru_file_url)
    if resp.status_code != 200:
        raise RuntimeError(f"Ошибка {resp.status_code} при запросе {mailru_file_url}")
    page = resp.text
    match = re.search(r'dispatcher.*?weblink_get.*?url":"(.*?)"', page)
    if not match:
        raise RuntimeError("Не удалось найти CDN ссылку в HTML")
    base_url = match.group(1)
    parts = mailru_file_url.split('/')[-3:]
    return f"{base_url}/{parts[0]}/{parts[1]}/{parts[2]}"

def download_from_mailru(file_url: str, local_name: str):
    """Скачивание файла с Mail.ru"""
    direct = get_direct_file_link(file_url)
    print(f"Скачиваем {file_url} → {local_name}")
    os.system(f"wget -q --content-disposition '{direct}' -O '{local_name}'")

# Автоматическая загрузка данных если их нет
if not os.path.exists(TRAIN_IMG_DIR) or len(os.listdir(TRAIN_IMG_DIR)) == 0:
    print("Скачиваем тренировочные изображения...")
    train_link = "https://cloud.mail.ru/public/2kaD/W4xWY9vgr/train_images.zip"
    download_from_mailru(train_link, "train_images.zip")
    print("Распаковываем тренировочные изображения...")
    os.system('unzip -q train_images.zip -d train_images && rm train_images.zip')

if not os.path.exists(TEST_IMG_DIR) or len(os.listdir(TEST_IMG_DIR)) == 0:
    print("Скачиваем тестовые изображения...")
    test_link = "https://cloud.mail.ru/public/2kaD/W4xWY9vgr/test_images.zip"
    download_from_mailru(test_link, "test_images.zip")
    print("Распаковываем тестовые изображения...")
    os.system('unzip -q test_images.zip -d test_images && rm test_images.zip')

# Загрузка табличных данных
print("Загружаем табличные данные...")
train_df = pd.read_parquet(TRAIN_PQ)
test_df = pd.read_parquet(TEST_PQ)

print(f"Train shape: {train_df.shape}, Test shape: {test_df.shape}")

# ==================== ПОДГОТОВКА ДАННЫХ ====================

def build_img_csv(df, img_dir, out_csv, n_images=4):
    """Создание CSV с путями к изображениям"""
    if os.path.exists(out_csv):
        print(f"Файл {out_csv} уже существует, загружаем...")
        return pd.read_csv(out_csv)

    print(f"Создаем {out_csv}...")
    all_files = os.listdir(img_dir)
    id_to_files = defaultdict(list)

    for fname in all_files:
        if fname.endswith(".jpg"):
            try:
                iid = int(fname.split("_")[0])
                id_to_files[iid].append(os.path.join(img_dir, fname))
            except ValueError:
                continue

    rows = []
    for r in df.itertuples(index=False):
        iid = getattr(r, "ID")
        paths = sorted(id_to_files.get(iid, []))[:n_images]

        row = {
            "item_id": iid,
            "paths": ";".join(paths)
        }

        if "price_TARGET" in df.columns:
            row["price"] = getattr(r, "price_TARGET")
        else:
            row["price"] = -1

        rows.append(row)

    csv_df = pd.DataFrame(rows)
    csv_df.to_csv(out_csv, index=False)
    print(f"CSV сохранён: {out_csv}, shape={csv_df.shape}")
    return csv_df

# Создаем CSV файлы с путями к изображениям
print("Создаем CSV файлы...")
train_csv_df = build_img_csv(train_df, TRAIN_IMG_DIR, TRAIN_CSV, n_images=N_IMAGES_PER_ITEM)
test_csv_df = build_img_csv(test_df, TEST_IMG_DIR, TEST_CSV, n_images=N_IMAGES_PER_ITEM)

# ==================== УЛУЧШЕННАЯ ПРЕДОБРАБОТКА ТАБЛИЧНЫХ ДАННЫХ ====================

class TabularPreprocessor:
    """Класс для обработки табличных данных"""

    def __init__(self):
        self.scaler = StandardScaler()
        self.label_encoders = {}
        self.feature_cols = []
        self.numeric_cols = ['doors_number', 'crashes_count', 'owners_count', 'mileage', 'latitude', 'longitude']
        self.categorical_cols = ['body_type', 'drive_type', 'engine_type', 'color', 'pts', 'steering_wheel']

    def fit_transform(self, train_df, test_df):
        """Обучение и преобразование данных"""
        print("🔧 Подготавливаем табличные данные...")

        train_processed = train_df.copy()
        test_processed = test_df.copy()

        # Удаляем сложные колонки
        cols_to_drop = ['close_date', 'equipment']
        train_processed = train_processed.drop(columns=cols_to_drop, errors='ignore')
        test_processed = test_processed.drop(columns=cols_to_drop, errors='ignore')

        # Обработка числовых колонок
        for col in self.numeric_cols:
            if col in train_processed.columns:
                train_processed[col] = pd.to_numeric(train_processed[col], errors='coerce')
                test_processed[col] = pd.to_numeric(test_processed[col], errors='coerce')

        # Кодируем категориальные признаки
        for col in self.categorical_cols:
            if col in train_processed.columns:
                print(f"📊 Кодируем {col}...")
                combined = pd.concat([train_processed[col], test_processed[col]]).fillna('Unknown')
                le = LabelEncoder()
                le.fit(combined.astype(str))
                self.label_encoders[col] = le

                train_processed[col] = le.transform(train_processed[col].fillna('Unknown').astype(str))
                test_processed[col] = le.transform(test_processed[col].fillna('Unknown').astype(str))

        # Мультивыборные поля
        multiselect_cols = [col for col in train_processed.columns if '_mult' in col]
        print(f"🎯 Мультивыборные поля: {len(multiselect_cols)}")

        for col in multiselect_cols:
            train_processed[f'{col}_count'] = train_processed[col].apply(
                lambda x: len(x) if isinstance(x, list) and x != [None] else 0
            )
            test_processed[f'{col}_count'] = test_processed[col].apply(
                lambda x: len(x) if isinstance(x, list) and x != [None] else 0
            )

        # Одновыборные опции
        single_select_cols = ['audiosistema', 'diski', 'electropodemniki', 'fary', 'salon', 'upravlenie_klimatom', 'usilitel_rul']
        single_select_cols = [col for col in single_select_cols if col in train_processed.columns]

        for col in single_select_cols:
            train_processed[col] = train_processed[col].notna().astype(int)
            test_processed[col] = test_processed[col].notna().astype(int)

        # Собираем финальные признаки
        self.feature_cols = (self.categorical_cols + self.numeric_cols +
                           [f'{col}_count' for col in multiselect_cols] +
                           single_select_cols)
        self.feature_cols = [col for col in self.feature_cols if col in train_processed.columns]

        print(f"✅ Всего признаков: {len(self.feature_cols)}")

        # Заполняем пропуски
        for col in self.feature_cols:
            if col in self.numeric_cols or '_count' in col:
                median_val = train_processed[col].median()
                train_processed[col] = train_processed[col].fillna(median_val)
                test_processed[col] = test_processed[col].fillna(median_val)
            else:
                mode_val = train_processed[col].mode()[0] if len(train_processed[col].mode()) > 0 else 0
                train_processed[col] = train_processed[col].fillna(mode_val)
                test_processed[col] = test_processed[col].fillna(mode_val)

        # Нормализация
        numeric_features = [col for col in self.feature_cols if col in self.numeric_cols or '_count' in col]
        if numeric_features:
            train_processed[numeric_features] = self.scaler.fit_transform(train_processed[numeric_features])
            test_processed[numeric_features] = self.scaler.transform(test_processed[numeric_features])

        return train_processed, test_processed

# Подготавливаем табличные данные
preprocessor = TabularPreprocessor()
train_tabular, test_tabular = preprocessor.fit_transform(train_df, test_df)
feature_cols = preprocessor.feature_cols

# ==================== УЛУЧШЕННЫЙ DATASET И АУГМЕНТАЦИИ ====================

# Более агрессивные аугментации для тренировки
train_tfms = A.Compose([
    A.LongestMaxSize(IMG_SIZE),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_CONSTANT, value=0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.RandomRotate90(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

valid_tfms = A.Compose([
    A.LongestMaxSize(IMG_SIZE),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_CONSTANT, value=0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

def _safe_imread(p: str, fallback_hw=(IMG_SIZE, IMG_SIZE)) -> np.ndarray:
    """Безопасное чтение изображения с кэшированием"""
    img = cv2.imread(p, cv2.IMREAD_COLOR)
    if img is None:
        h, w = fallback_hw
        img = np.zeros((h, w, 3), dtype=np.uint8)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

class ImprovedCarsDataset(Dataset):
    """Улучшенный датасет с кэшированием и оптимизацией памяти"""

    def __init__(self, csv_df: pd.DataFrame, tabular_df: pd.DataFrame, feature_cols: list, is_train: bool = True):
        self.csv_df = csv_df.reset_index(drop=True)
        self.tabular_df = tabular_df.reset_index(drop=True)
        self.feature_cols = feature_cols
        self.tfms = train_tfms if is_train else valid_tfms
        self.has_target = "price" in csv_df.columns
        self.is_train = is_train

        # Предварительно вычисляем табличные признаки
        self.tabular_features = {}
        for idx, row in self.csv_df.iterrows():
            item_id = row["item_id"]
            tabular_mask = self.tabular_df['ID'] == item_id
            if tabular_mask.any():
                tabular_row = self.tabular_df[tabular_mask].iloc[0]
                self.tabular_features[idx] = tabular_row[self.feature_cols].values.astype(np.float32)
            else:
                self.tabular_features[idx] = np.zeros(len(feature_cols), dtype=np.float32)

    def __len__(self):
        return len(self.csv_df)

    def __getitem__(self, idx: int):
        row = self.csv_df.iloc[idx]
        item_id = row["item_id"]

        # Табличные признаки (предварительно вычислены)
        tabular_features = torch.tensor(self.tabular_features[idx], dtype=torch.float32)

        # Обработка изображений
        paths = [p for p in str(row["paths"]).split(";") if p and p.lower() != "nan"][:N_IMAGES_PER_ITEM]
        imgs = []

        for p in paths:
            img = _safe_imread(p)
            img = self.tfms(image=img)["image"]
            imgs.append(img)

        # Добиваем до нужного количества изображений
        while len(imgs) < N_IMAGES_PER_ITEM:
            imgs.append(torch.zeros(3, IMG_SIZE, IMG_SIZE))

        imgs = torch.stack(imgs, dim=0)

        if self.has_target:
            y = torch.tensor(row["price"], dtype=torch.float32)
            if USE_LOG_TARGET:
                y = torch.log1p(y)
            return imgs, tabular_features, y, torch.tensor(row["price"], dtype=torch.float32)  # Добавляем исходную цену
        else:
            return imgs, tabular_features, torch.tensor(item_id, dtype=torch.long)

# ==================== УЛУЧШЕННАЯ МОДЕЛЬ ====================

class ImprovedMultiImageResNet(nn.Module):
    """Улучшенная модель с оптимизированной архитектурой"""

    def __init__(self, tabular_input_size=0, use_tabular=True, dropout_rate=0.3):
        super().__init__()
        self.use_tabular = use_tabular

        # Визуальная часть с замороженными начальными слоями
        backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        in_features_visual = backbone.fc.in_features

        # Замораживаем начальные слои
        for param in list(backbone.parameters())[:50]:
            param.requires_grad = False

        self.visual_backbone = nn.Sequential(*list(backbone.children())[:-1])
        self.visual_pool = nn.AdaptiveAvgPool2d(1)

        # Улучшенный attention механизм
        self.attention = nn.Sequential(
            nn.Linear(in_features_visual, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 1)
        )

        # Улучшенная табличная часть
        if use_tabular:
            self.tabular_net = nn.Sequential(
                nn.Linear(tabular_input_size, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout_rate),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout_rate/2),
                nn.Linear(256, 128),
            )
            combined_features = in_features_visual + 128
        else:
            combined_features = in_features_visual

        # Улучшенная голова
        self.head = nn.Sequential(
            nn.Linear(combined_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate/2),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 1)
        )

        # Инициализация весов
        self._initialize_weights()

    def _initialize_weights(self):
        """Инициализация весов новых слоев"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, images, tabular_features=None):
        B, N, C, H, W = images.shape

        # Обработка изображений
        images_flat = images.view(B * N, C, H, W)
        visual_feats = self.visual_backbone(images_flat)
        visual_feats = self.visual_pool(visual_feats).view(B * N, -1)
        visual_feats = visual_feats.view(B, N, -1)

        # Attention с температурой для смягчения softmax
        attention_weights = self.attention(visual_feats) / np.sqrt(visual_feats.size(-1))
        attention_weights = F.softmax(attention_weights, dim=1)
        weighted_visual = (visual_feats * attention_weights).sum(dim=1)

        # Табличные данные
        if self.use_tabular and tabular_features is not None:
            tabular_feats = self.tabular_net(tabular_features)
            combined = torch.cat([weighted_visual, tabular_feats], dim=1)
        else:
            combined = weighted_visual

        output = self.head(combined)
        return output.squeeze(1)

# ==================== УЛУЧШЕННЫЕ ФУНКЦИИ ПОТЕРЬ ====================

class ImprovedCombinedLoss(nn.Module):
    """Улучшенная комбинированная функция потерь"""

    def __init__(self, alpha=0.8, smooth_l1_beta=1.0):
        super().__init__()
        self.alpha = alpha
        self.smooth_l1 = nn.SmoothL1Loss(beta=smooth_l1_beta)
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        smooth_l1_loss = self.smooth_l1(y_pred, y_true)
        mse_loss = self.mse(y_pred, y_true)
        return self.alpha * smooth_l1_loss + (1 - self.alpha) * mse_loss

class RMSLELoss(nn.Module):
    """RMSLE Loss для регрессии цен"""
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(torch.log1p(y_pred), torch.log1p(y_true)))

# ==================== УЛУЧШЕННОЕ ОБУЧЕНИЕ ====================

def run_epoch(model, loader, optimizer=None, device="cuda", loss_fn=None, scheduler=None):
    """Улучшенная функция эпохи с градиентным накоплением"""
    is_train = optimizer is not None
    model.train(is_train)

    losses = []
    all_preds, all_targets = [], []

    # Градиентное накопление
    accumulation_steps = 4
    pbar = tqdm(loader, desc="Train" if is_train else "Valid", leave=False)

    for i, batch in enumerate(pbar):
        if len(batch) == 4:  # train/val
            imgs, tabular, y_log, y_original = batch
        else:  # test
            imgs, tabular, ids = batch
            y_log = y_original = None

        imgs = imgs.to(device, non_blocking=True)
        tabular = tabular.to(device, non_blocking=True)

        if is_train and y_log is not None:
            y_log = y_log.to(device, non_blocking=True)

            with torch.cuda.amp.autocast():
                preds_log = model(imgs, tabular)
                loss = loss_fn(preds_log, y_log) / accumulation_steps

            loss.backward()

            if (i + 1) % accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                if scheduler:
                    scheduler.step()

            losses.append(loss.item() * accumulation_steps)

        elif not is_train and y_log is not None:
            y_log = y_log.to(device, non_blocking=True)
            y_original = y_original.to(device, non_blocking=True)

            with torch.no_grad():
                preds_log = model(imgs, tabular)
                loss = loss_fn(preds_log, y_log)
                losses.append(loss.item())

                # Для метрик преобразуем обратно
                preds_original = torch.expm1(preds_log)
                all_preds.append(preds_original.cpu())
                all_targets.append(y_original.cpu())

        pbar.set_postfix({"loss": f"{loss.item():.3f}"})

    if is_train:
        return float(np.mean(losses)) if losses else 0.0
    else:
        if all_preds:
            all_preds = torch.cat(all_preds).numpy()
            all_targets = torch.cat(all_targets).numpy()
            medape = median_absolute_percentage_error(all_targets, all_preds) * 100.0
            return float(np.mean(losses)), float(medape)
        return 0.0, 0.0

def median_absolute_percentage_error(y_true, y_pred):
    """Вычисление медианной абсолютной процентной ошибки"""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    epsilon = 1e-6
    ape = 2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred) + epsilon)
    return float(np.median(ape))

# ==================== ОСНОВНАЯ ФУНКЦИЯ ОБУЧЕНИЯ ====================

def main_training():
    """Улучшенная функция обучения"""
    print("🚀 Запускаем улучшенное обучение...")

    # Подготовка данных
    df = pd.read_csv(TRAIN_CSV)
    trn_df, val_df = train_test_split(df, test_size=VAL_SIZE, random_state=SEED, shuffle=True)

    print(f"📊 Train samples: {len(trn_df)}, Val samples: {len(val_df)}")

    # Создаем датасеты
    train_ds = ImprovedCarsDataset(trn_df, train_tabular, feature_cols, is_train=True)
    valid_ds = ImprovedCarsDataset(val_df, train_tabular, feature_cols, is_train=False)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                             pin_memory=True, num_workers=4, drop_last=True,
                             persistent_workers=True)
    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False,
                             pin_memory=True, num_workers=4, drop_last=False,
                             persistent_workers=True)

    # Создаем модель
    model = ImprovedMultiImageResNet(
        tabular_input_size=len(feature_cols),
        use_tabular=True,
        dropout_rate=0.3
    ).to(device)

    print(f"🧠 Модель создана. Параметров: {sum(p.numel() for p in model.parameters()):,}")
    print(f"📈 Обучаемых параметров: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

    # Функция потерь и оптимизатор
    loss_fn = ImprovedCombinedLoss(alpha=0.7)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, betas=(0.9, 0.999))
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

    # Automatic Mixed Precision
    scaler = torch.cuda.amp.GradScaler()

    history = {
        "epoch": [], "train_loss": [], "val_loss": [],
        "val_medape": [], "lr": []
    }

    best_val_medape = float("inf")
    patience = 10
    patience_counter = 0

    print(f"🎯 Начинаем обучение на {len(train_ds)} примерах...")

    for epoch in range(1, EPOCHS + 1):
        print(f"\n📍 Эпоха {epoch}/{EPOCHS}")

        # Обучение
        train_loss = run_epoch(model, train_loader, optimizer, device, loss_fn, scheduler)

        # Валидация
        val_loss, val_medape = run_epoch(model, valid_loader, None, device, loss_fn)

        current_lr = optimizer.param_groups[0]['lr']

        print(f"📉 Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MedAPE: {val_medape:.2f}%")
        print(f"📚 Learning Rate: {current_lr:.2e}")

        # Сохраняем историю
        history["epoch"].append(epoch)
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["val_medape"].append(val_medape)
        history["lr"].append(current_lr)

        # Ранняя остановка
        if val_medape < best_val_medape:
            best_val_medape = val_medape
            patience_counter = 0

            torch.save({
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scheduler": scheduler.state_dict(),
                "epoch": epoch,
                "val_medape": val_medape,
                "history": history,
                "feature_cols": feature_cols
            }, CKPT_PATH)
            print(f"💾 Лучшая модель сохранена в эпоху {epoch} -> {CKPT_PATH}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"🛑 Ранняя остановка на эпохе {epoch}")
                break

    # Визуализация результатов
    plot_training_history(history)

    return model, history

def plot_training_history(history):
    """Визуализация истории обучения"""
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.plot(history["epoch"], history["val_medape"], 'r-', label="Validation MedAPE")
    plt.xlabel("Epoch")
    plt.ylabel("Median APE, %")
    plt.title("Validation Median APE")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 2)
    plt.plot(history["epoch"], history["train_loss"], 'b-', label="Train Loss")
    plt.plot(history["epoch"], history["val_loss"], 'r-', label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 3)
    plt.plot(history["epoch"], history["lr"], 'g-')
    plt.xlabel("Epoch")
    plt.ylabel("Learning Rate")
    plt.title("Learning Rate Schedule")
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(f"improved_training_history.png", dpi=150, bbox_inches='tight')
    plt.show()

    # Сохраняем историю
    pd.DataFrame(history).to_csv(f"{CKPT_NAME}_training_history.csv", index=False)

# ==================== УЛУЧШЕННЫЙ ИНФЕРЕНС ====================

def create_final_submission(model_path=None):
    """Создание финального сабмита"""
    print("🎯 Создаем финальный сабмит...")

    if model_path is None:
        model_path = CKPT_PATH

    # Загружаем чекпоинт
    checkpoint = torch.load(model_path, map_location=device)

    model = ImprovedMultiImageResNet(
        tabular_input_size=len(feature_cols),
        use_tabular=True
    ).to(device)

    model.load_state_dict(checkpoint['model'])
    model.eval()

    # Тестовый датасет
    test_ds = ImprovedCarsDataset(test_csv_df, test_tabular, feature_cols, is_train=False)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE * 2, shuffle=False,
                            num_workers=4, pin_memory=True, drop_last=False)

    # Предсказания
    predictions_dict = {}

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Test Inference"):
            if len(batch) == 3:  # test data
                imgs, tabular, ids = batch
                imgs = imgs.to(device, non_blocking=True)
                tabular = tabular.to(device, non_blocking=True)

                preds_log = model(imgs, tabular)
                preds = torch.expm1(preds_log) if USE_LOG_TARGET else preds_log

                batch_ids = ids.cpu().numpy()
                batch_preds = preds.cpu().numpy()

                for item_id, pred in zip(batch_ids, batch_preds):
                    predictions_dict[item_id] = pred

    # Создаем сабмит
    submission_rows = []
    for item_id in test_df['ID'].values:
        pred = predictions_dict.get(item_id, np.mean(list(predictions_dict.values())))
        submission_rows.append({'ID': item_id, 'price_TARGET': pred})

    submission_df = pd.DataFrame(submission_rows)
    submission_file = f"final_submission_{CKPT_NAME}.csv"
    submission_df.to_csv(submission_file, index=False)

    print(f"✅ Финальный сабмит сохранен: {submission_file}")
    print(f"📊 Статистика предсказаний:")
    print(f"   - Минимальная цена: {submission_df['price_TARGET'].min():.2f} руб.")
    print(f"   - Максимальная цена: {submission_df['price_TARGET'].max():.2f} руб.")
    print(f"   - Медианная цена: {submission_df['price_TARGET'].median():.2f} руб.")
    print(f"   - Средняя цена: {submission_df['price_TARGET'].mean():.2f} руб.")

    return submission_df

# ==================== ЗАПУСК ====================

if __name__ == "__main__":
    try:
        # Очищаем память
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # Обучаем модель
        trained_model, training_history = main_training()

        # Создаем сабмит
        submission = create_final_submission()

        print("\n" + "="*60)
        print("🎉 ОБУЧЕНИЕ УСПЕШНО ЗАВЕРШЕНО!")
        print(f"🏆 Лучшая валидационная MedAPE: {min(training_history['val_medape']):.2f}%")
        print(f"💾 Модель сохранена: {CKPT_PATH}")
        print(f"📄 Сабмит готов: final_submission_{CKPT_NAME}.csv")
        print("="*60)

    except Exception as e:
        print(f"\n❌ Ошибка: {e}")
        import traceback
        traceback.print_exc()

Device: cuda
Загружаем табличные данные...
Train shape: (70000, 35), Test shape: (25000, 34)
Создаем CSV файлы...
Создаем train.csv...
CSV сохранён: train.csv, shape=(70000, 3)
Создаем test.csv...
CSV сохранён: test.csv, shape=(25000, 3)
🔧 Подготавливаем табличные данные...
📊 Кодируем body_type...
📊 Кодируем drive_type...
📊 Кодируем engine_type...
📊 Кодируем color...
📊 Кодируем pts...
📊 Кодируем steering_wheel...
🎯 Мультивыборные поля: 13
✅ Всего признаков: 32
🚀 Запускаем улучшенное обучение...
📊 Train samples: 59500, Val samples: 10500
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 168MB/s]


🧠 Модель создана. Параметров: 12,018,114
📈 Обучаемых параметров: 5,694,850
🎯 Начинаем обучение на 59500 примерах...

📍 Эпоха 1/20



❌ Ошибка: running_mean should contain 4 elements not 256


Traceback (most recent call last):
  File "/tmp/ipython-input-3261624057.py", line 743, in <cell line: 0>
    trained_model, training_history = main_training()
                                      ^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-3261624057.py", line 593, in main_training
    train_loss = run_epoch(model, train_loader, optimizer, device, loss_fn, scheduler)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-3261624057.py", line 489, in run_epoch
    preds_log = model(imgs, tabular)
                ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 1773, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 1784, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.decomposition import PCA
import xgboost as xgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50, ResNet50_Weights
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# =============================================================================

def median_absolute_percentage_error(y_true, y_pred):
    """Вычисляет median absolute percentage error (medianAPE)"""
    mask = y_true > 0
    if mask.sum() == 0:
        return 1.0  # Если все истинные значения <= 0, возвращаем худший результат
    ape = np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]
    return float(np.median(ape))

# =============================================================================
# ЗАГРУЗКА ДАННЫХ
# =============================================================================

print("=" * 50)
print("ЗАГРУЗКА ДАННЫХ")
print("=" * 50)

# Загрузка табличных данных
train_df = pd.read_parquet('/content/train_dataset.parquet')
test_df = pd.read_parquet('/content/test_dataset.parquet')
print(f"Тренировочные данные: {train_df.shape}")
print(f"Тестовые данные: {test_df.shape}")

# Проверка наличия папок с изображениями
TRAIN_IMG_DIR = '/content/train_images'
TEST_IMG_DIR = '/content/test_images'

print(f"Папка train_images существует: {os.path.exists(TRAIN_IMG_DIR)}")
print(f"Папка test_images существует: {os.path.exists(TEST_IMG_DIR)}")

# =============================================================================
# ДАТАСЕТ И МОДЕЛЬ ДЛЯ ДЕТЕКЦИИ ПОВРЕЖДЕНИЙ
# =============================================================================

class SafeCarDamageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None, target_prices=None, img_size=224):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform
        self.target_prices = target_prices
        self.img_size = img_size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        car_id = self.df.iloc[idx]['ID']

        # Ищем первое доступное изображение
        image_path = None
        for i in range(4):  # пробуем 0-3 изображения
            potential_path = f"{self.image_dir}/{car_id}_{i}.jpg"
            if os.path.exists(potential_path):
                image_path = potential_path
                break

        # Загружаем изображение
        if image_path:
            try:
                image = cv2.imread(image_path)
                if image is None:
                    raise ValueError(f"Не удалось загрузить изображение: {image_path}")
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            except Exception as e:
                print(f"Ошибка загрузки {image_path}: {e}")
                image = np.ones((self.img_size, self.img_size, 3), dtype=np.uint8) * 128
        else:
            # Создаем серое изображение если файл не найден
            image = np.ones((self.img_size, self.img_size, 3), dtype=np.uint8) * 128

        # Гарантируем минимальный размер изображения
        h, w = image.shape[:2]
        if h < self.img_size or w < self.img_size:
            # Увеличиваем маленькие изображения
            scale = max(self.img_size / h, self.img_size / w)
            new_h, new_w = int(h * scale), int(w * scale)
            image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

        # Применяем трансформации
        if self.transform:
            try:
                image = self.transform(image=image)['image']
            except Exception as e:
                print(f"Ошибка трансформации для car_id {car_id}: {e}")
                # Создаем дефолтное изображение
                image = torch.ones(3, self.img_size, self.img_size) * 0.5

        # Создаем target для детекции повреждений
        if self.target_prices is not None:
            price = self.target_prices.iloc[idx]
            # Считаем автомобиль "убитым" если цена в нижних 15%
            is_damaged = 1.0 if price < np.percentile(self.target_prices, 15) else 0.0
            return image, torch.tensor(is_damaged, dtype=torch.float32)

        return image, torch.tensor(0.0, dtype=torch.float32)  # заглушка для теста

class DamageDetector(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        self.backbone = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()  # убираем классификатор

        # Детектор повреждений
        self.damage_classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes),
            nn.Sigmoid() if num_classes == 1 else nn.Softmax(dim=1)
        )

        # Регрессор для извлечения features
        self.feature_extractor = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 64)  # компактные фичи для XGBoost
        )

    def forward(self, x, return_features=False):
        features = self.backbone(x)
        damage_score = self.damage_classifier(features)

        if return_features:
            compact_features = self.feature_extractor(features)
            return damage_score, compact_features

        return damage_score

def get_safe_transforms(mode='train', img_size=224):
    """Безопасные трансформации для изображений любого размера"""
    if mode == 'train':
        return A.Compose([
            A.LongestMaxSize(img_size * 2),  # Сначала увеличиваем
            A.PadIfNeeded(
                min_height=img_size,
                min_width=img_size,
                border_mode=cv2.BORDER_CONSTANT,
                value=0
            ),
            A.RandomCrop(img_size, img_size, p=0.8),
            A.Resize(img_size, img_size, p=1.0),  # Гарантируем размер
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.GaussNoise(p=0.2),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])
    else:  # validation/test
        return A.Compose([
            A.LongestMaxSize(img_size * 2),
            A.PadIfNeeded(
                min_height=img_size,
                min_width=img_size,
                border_mode=cv2.BORDER_CONSTANT,
                value=0
            ),
            A.CenterCrop(img_size, img_size),
            A.Resize(img_size, img_size),  # Гарантируем размер
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])

def train_damage_detector(train_df, image_dir, device='cuda', img_size=224):
    """Обучает детектор поврежденных автомобилей"""
    print("Обучение детектора повреждений...")

    # Безопасные трансформации
    train_transform = get_safe_transforms('train', img_size)

    # Создаем датасет
    dataset = SafeCarDamageDataset(
        train_df,
        image_dir,
        transform=train_transform,
        target_prices=train_df['price_TARGET'],
        img_size=img_size
    )

    # DataLoader с обработкой ошибок
    dataloader = DataLoader(
        dataset,
        batch_size=16,  # Уменьшил batch_size для стабильности
        shuffle=True,
        num_workers=2,   # Уменьшил для стабильности
        pin_memory=True,
        persistent_workers=True
    )

    # Модель и оптимизатор
    model = DamageDetector().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

    # Обучение
    model.train()
    for epoch in range(3):  # Уменьшил эпохи для скорости
        total_loss = 0
        for batch_idx, (images, targets) in enumerate(dataloader):
            try:
                images, targets = images.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs.squeeze(), targets)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

                if batch_idx % 50 == 0:
                    print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}')

            except Exception as e:
                print(f'Ошибка в батче {batch_idx}: {e}')
                continue

        scheduler.step()
        print(f'Epoch {epoch} completed. Average Loss: {total_loss/len(dataloader):.4f}')

    return model

def extract_damage_features(model, df, image_dir, device='cuda', img_size=224):
    """Извлекает фичи повреждений и эмбеддинги"""
    print("Извлечение признаков повреждений...")

    # Безопасные трансформации для инференса
    transform = get_safe_transforms('val', img_size)

    dataset = SafeCarDamageDataset(df, image_dir, transform=transform, img_size=img_size)
    dataloader = DataLoader(
        dataset,
        batch_size=32,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    model.eval()
    damage_scores = []
    damage_features = []

    with torch.no_grad():
        for images, _ in dataloader:
            try:
                images = images.to(device)
                scores, features = model(images, return_features=True)

                damage_scores.extend(scores.cpu().numpy())
                damage_features.extend(features.cpu().numpy())
            except Exception as e:
                print(f"Ошибка при обработке батча: {e}")
                # Добавляем нулевые фичи для проблемного батча
                batch_size = images.size(0) if hasattr(images, 'size') else 32
                damage_scores.extend([0.0] * batch_size)
                damage_features.extend([np.zeros(64)] * batch_size)

    return np.array(damage_scores).flatten(), np.array(damage_features)

# =============================================================================
# ИСПРАВЛЕННАЯ ПРЕДОБРАБОТКА (без category dtype)
# =============================================================================

def enhanced_preprocess_with_damage(df, damage_scores, damage_features, is_train=True):
    """Улучшенная предобработка с учетом обученных признаков повреждений"""
    df_processed = df.copy()

    # 1. Добавляем предсказанный score повреждений
    df_processed['damage_score'] = damage_scores

    # 2. Создаем бинарные признаки на основе обученной модели
    if is_train and len(damage_scores) > 0:
        damage_threshold = np.percentile(damage_scores, 20)  # 20% самых "убитых"
        well_maintained_threshold = np.percentile(damage_scores, 80)  # 20% самых ухоженных
    else:
        damage_threshold = 0.7  # консервативный порог
        well_maintained_threshold = 0.3

    df_processed['is_severely_damaged'] = (df_processed['damage_score'] > damage_threshold).astype(int)
    df_processed['is_well_maintained'] = (df_processed['damage_score'] < well_maintained_threshold).astype(int)

    # ИСПРАВЛЕНИЕ: вместо category dtype используем числовое кодирование
    damage_bins = [0, 0.3, 0.7, 1.0]
    damage_labels = [0, 1, 2]  # good=0, average=1, damaged=2
    df_processed['damage_category'] = pd.cut(
        df_processed['damage_score'],
        bins=damage_bins,
        labels=damage_labels
    ).astype(int)  # Преобразуем в int

    # 3. Обработка числовых признаков с учетом повреждений
    numeric_columns = ['mileage', 'crashes_count', 'owners_count', 'latitude', 'longitude']

    for col in numeric_columns:
        if col in df_processed.columns:
            df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
            # Заполняем медианами сгруппированными по damage_category
            if is_train:
                for category in df_processed['damage_category'].unique():
                    mask = df_processed['damage_category'] == category
                    median_val = df_processed.loc[mask, col].median()
                    df_processed.loc[mask, col] = df_processed.loc[mask, col].fillna(median_val)
            else:
                df_processed[col] = df_processed[col].fillna(df_processed[col].median())

    # 4. Feature engineering с взаимодействиями
    if 'mileage' in df_processed.columns:
        df_processed['log_mileage'] = np.log1p(df_processed['mileage'])
        df_processed['mileage_damage_interaction'] = df_processed['mileage'] * df_processed['damage_score']
        df_processed['high_mileage_damaged'] = (
            (df_processed['mileage'] > df_processed['mileage'].median()) &
            (df_processed['is_severely_damaged'] == 1)
        ).astype(int)

    if 'crashes_count' in df_processed.columns:
        df_processed['has_crashes'] = (df_processed['crashes_count'] > 0).astype(int)
        df_processed['crashes_damage_interaction'] = df_processed['crashes_count'] * df_processed['damage_score']
        df_processed['crashed_and_damaged'] = (
            (df_processed['has_crashes'] == 1) &
            (df_processed['is_severely_damaged'] == 1)
        ).astype(int)

    if 'owners_count' in df_processed.columns:
        df_processed['single_owner'] = (df_processed['owners_count'] == 1).astype(int)
        df_processed['many_owners'] = (df_processed['owners_count'] > 3).astype(int)
        df_processed['multiple_owners_damaged'] = (
            (df_processed['many_owners'] == 1) &
            (df_processed['is_severely_damaged'] == 1)
        ).astype(int)

    # 5. Категориальные признаки - преобразуем в строки
    categorical_columns = ['body_type', 'drive_type', 'engine_type', 'color', 'pts', 'steering_wheel']

    for col in categorical_columns:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna('unknown').astype(str)

    # 6. Однозначные опции - преобразуем в строки
    single_select_columns = [
        'audiosistema', 'diski', 'electropodemniki', 'fary', 'salon',
        'upravlenie_klimatom', 'usilitel_rul'
    ]

    for col in single_select_columns:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna('unknown').astype(str)

    # 7. Мультивыборные поля
    multiselect_columns = [col for col in df_processed.columns if '_mult' in col]

    for col in multiselect_columns:
        if col in df_processed.columns:
            try:
                df_processed[f'{col}_count'] = df_processed[col].apply(
                    lambda x: len(x) if isinstance(x, list) else (1 if pd.notna(x) and x != 'None' else 0)
                )
                # Взаимодействие с повреждениями
                df_processed[f'{col}_damaged'] = (
                    (df_processed[f'{col}_count'] > 0) &
                    (df_processed['is_severely_damaged'] == 1)
                ).astype(int)
            except:
                df_processed[f'{col}_count'] = 0
                df_processed[f'{col}_damaged'] = 0

    for col in multiselect_columns:
        if col in df_processed.columns:
            df_processed = df_processed.drop(col, axis=1)

    # 8. Добавляем обученные фичи повреждений
    for i in range(damage_features.shape[1]):
        df_processed[f'damage_feature_{i}'] = damage_features[:, i]

    # 9. Убеждаемся, что все числовые колонки имеют правильный тип
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')

    return df_processed

# =============================================================================
# УЛУЧШЕННЫЙ XGBOOST С ПРАВИЛЬНОЙ ПОДГОТОВКОЙ ДАННЫХ
# =============================================================================

def prepare_data_for_xgboost(X, X_test):
    """Подготавливает данные для XGBoost, преобразуя все в числовые типы"""
    print("Подготовка данных для XGBoost...")

    # Создаем копии
    X_encoded = X.copy()
    X_test_encoded = X_test.copy()

    # 1. Идентифицируем категориальные колонки
    categorical_columns = [col for col in X_encoded.columns if X_encoded[col].dtype == 'object']
    print(f"Найдено категориальных колонок: {len(categorical_columns)}")

    # 2. Label Encoding для категориальных признаков
    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        # Объединяем train и test для консистентного кодирования
        combined = pd.concat([X_encoded[col], X_test_encoded[col]], axis=0)
        le.fit(combined.astype(str))
        X_encoded[col] = le.transform(X_encoded[col].astype(str))
        X_test_encoded[col] = le.transform(X_test_encoded[col].astype(str))
        label_encoders[col] = le

    # 3. Проверяем типы данных
    print("Проверка типов данных после кодирования:")
    print(X_encoded.dtypes.value_counts())

    # 4. Масштабирование числовых признаков
    numeric_columns = X_encoded.select_dtypes(include=[np.number]).columns
    print(f"Числовых колонок для масштабирования: {len(numeric_columns)}")

    scaler = StandardScaler()
    X_scaled = X_encoded.copy()
    X_test_scaled = X_test_encoded.copy()

    X_scaled[numeric_columns] = scaler.fit_transform(X_encoded[numeric_columns])
    X_test_scaled[numeric_columns] = scaler.transform(X_test_encoded[numeric_columns])

    # 5. Финальная проверка типов
    for col in X_scaled.columns:
        if X_scaled[col].dtype.name == 'category':
            X_scaled[col] = X_scaled[col].astype(int)
        if X_test_scaled[col].dtype.name == 'category':
            X_test_scaled[col] = X_test_scaled[col].astype(int)

    print("Финальные типы данных:")
    print(X_scaled.dtypes.value_counts())

    return X_scaled, X_test_scaled, label_encoders, scaler

def train_xgboost_with_validation(X, y, X_test):
    """Обучает XGBoost с валидацией"""
    print("Обучение XGBoost...")

    # Разделение на train/val
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.15, random_state=42, shuffle=True
    )

    # Параметры XGBoost
    params = {
        'n_estimators': 2000,
        'max_depth': 8,
        'learning_rate': 0.005,
        'subsample': 0.9,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.1,
        'reg_lambda': 1.0,
        'random_state': 42,
        'n_jobs': -1,
        'eval_metric': 'mae',
        'early_stopping_rounds': 350
    }

    model = xgb.XGBRegressor(**params)

    print("Начало обучения XGBoost...")
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=100
    )

    # Предсказание на валидации
    y_pred = model.predict(X_val)
    score = median_absolute_percentage_error(y_val, y_pred)
    print(f"XGBoost medianAPE: {score:.4f}")

    # Предсказание на тесте
    test_predictions = model.predict(X_test)

    return model, test_predictions, score

# =============================================================================
# ПОЛНЫЙ ПАЙПЛАЙН С ОБУЧЕНИЕМ CNN
# =============================================================================

def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Используемое устройство: {device}")

    try:
        # 1. Обучаем детектор повреждений
        print("=" * 50)
        print("ОБУЧЕНИЕ CNN ДЕТЕКТОРА ПОВРЕЖДЕНИЙ")
        print("=" * 50)
        damage_model = train_damage_detector(train_df, TRAIN_IMG_DIR, device, img_size=224)

        # 2. Извлекаем признаки повреждений
        print("\n" + "=" * 50)
        print("ИЗВЛЕЧЕНИЕ ПРИЗНАКОВ ПОВРЕЖДЕНИЙ")
        print("=" * 50)
        train_damage_scores, train_damage_features = extract_damage_features(
            damage_model, train_df, TRAIN_IMG_DIR, device, img_size=224
        )
        test_damage_scores, test_damage_features = extract_damage_features(
            damage_model, test_df, TEST_IMG_DIR, device, img_size=224
        )

        print(f"Признаки повреждений train: {train_damage_features.shape}")
        print(f"Признаки повреждений test: {test_damage_features.shape}")

    except Exception as e:
        print(f"Ошибка при работе с изображениями: {e}")
        print("Создаем случайные признаки повреждений как fallback...")
        # Создаем случайные фичи как fallback
        train_damage_scores = np.random.uniform(0, 1, len(train_df))
        test_damage_scores = np.random.uniform(0, 1, len(test_df))
        train_damage_features = np.random.normal(0, 1, (len(train_df), 64))
        test_damage_features = np.random.normal(0, 1, (len(test_df), 64))

    # 3. Предобработка с учетом повреждений
    print("\n" + "=" * 50)
    print("ПРЕДОБРАБОТКА ДАННЫХ")
    print("=" * 50)
    train_processed = enhanced_preprocess_with_damage(
        train_df, train_damage_scores, train_damage_features, is_train=True
    )
    test_processed = enhanced_preprocess_with_damage(
        test_df, test_damage_scores, test_damage_features, is_train=False
    )

    print(f"Обработанные train данные: {train_processed.shape}")
    print(f"Обработанные test данные: {test_processed.shape}")

    # 4. Подготовка данных для XGBoost
    common_columns = list(set(train_processed.columns) & set(test_processed.columns))
    if 'price_TARGET' in common_columns:
        common_columns.remove('price_TARGET')

    X = train_processed[common_columns]
    y = train_processed['price_TARGET']
    X_test = test_processed[common_columns]

    print(f"\nДанные для обучения: {X.shape}")
    print(f"Данные для теста: {X_test.shape}")

    # 5. Подготовка данных для XGBoost
    X_prepared, X_test_prepared, label_encoders, scaler = prepare_data_for_xgboost(X, X_test)

    # 6. Обучение XGBoost
    print("\n" + "=" * 50)
    print("ОБУЧЕНИЕ XGBOOST")
    print("=" * 50)
    model, test_predictions, score = train_xgboost_with_validation(X_prepared, y, X_test_prepared)

    # 7. Постобработка предсказаний
    print("\n" + "=" * 50)
    print("ПОСТОБРАБОТКА ПРЕДСКАЗАНИЙ")
    print("=" * 50)

    # Корректировка цен для поврежденных автомобилей
    damage_adjustment = 0.15  # снижаем на 15% для сильно поврежденных
    severely_damaged_mask = test_processed['is_severely_damaged'] == 1
    test_predictions[severely_damaged_mask] = test_predictions[severely_damaged_mask] * (1 - damage_adjustment)

    print(f"Скорректировано цен для {severely_damaged_mask.sum()} поврежденных автомобилей")

    # Обрезка выбросов
    q1 = np.percentile(test_predictions, 1)
    q99 = np.percentile(test_predictions, 99)
    test_predictions = np.clip(test_predictions, q1, q99)

    print(f"Диапазон предсказаний: {test_predictions.min():.0f} - {test_predictions.max():.0f}")

    # 8. Создание сабмишена
    submission = pd.DataFrame({
        'ID': test_df['ID'],
        'target': test_predictions
    })

    submission.to_csv('xgboost_with_cnn_damage_detector.csv', index=False)
    print("\nСабмишн сохранен в xgboost_with_cnn_damage_detector.csv")

    # 9. Анализ важности признаков
    print("\n" + "=" * 50)
    print("АНАЛИЗ ВАЖНОСТИ ПРИЗНАКОВ")
    print("=" * 50)
    feature_importance = pd.DataFrame({
        'feature': X_prepared.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nТоп-15 важных признаков:")
    print(feature_importance.head(15))

    # Анализ признаков повреждений
    damage_related_features = [col for col in feature_importance['feature']
                              if 'damage' in col.lower() or 'feature_' in col]
    print(f"\nПризнаков повреждений в топе: {len(damage_related_features)}")
    if damage_related_features:
        print("Важные признаки повреждений:")
        for feat in damage_related_features[:10]:
            imp = feature_importance[feature_importance['feature'] == feat]['importance'].values[0]
            print(f"  {feat}: {imp:.4f}")

    return submission, model, feature_importance

if __name__ == "__main__":
    submission, model, feature_importance = main()
    print("\n🎯 Пайплайн с обучением CNN успешно завершен!")

ЗАГРУЗКА ДАННЫХ
Тренировочные данные: (70000, 35)
Тестовые данные: (25000, 34)
Папка train_images существует: True
Папка test_images существует: True
Используемое устройство: cuda
ОБУЧЕНИЕ CNN ДЕТЕКТОРА ПОВРЕЖДЕНИЙ
Обучение детектора повреждений...
